In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 사용 패키지

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
import datetime

# 데이터 로드

In [2]:
path = "/content/drive/MyDrive/Colab Notebooks/cw_daicon/gas/"

In [5]:
total = pd.read_csv(path+'20181231.csv', encoding='cp949')

In [ ]:
total.head()

,연월일,시간,구분,공급량
0,2013-01-01,1,A,2497.129
1,2013-01-01,2,A,2363.265
2,2013-01-01,3,A,2258.505
3,2013-01-01,4,A,2243.969
4,2013-01-01,5,A,2344.105


# 전처리

In [ ]:
total['구분'].unique()

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

In [6]:
d_map = {}
for i, d in enumerate(total['구분'].unique()):
    d_map[d] = i
total['구분'] = total['구분'].map(d_map)

In [7]:
total['연월일'] = pd.to_datetime(total['연월일'])

In [8]:
total['year'] = total['연월일'].dt.year
total['month'] = total['연월일'].dt.month
total['day'] = total['연월일'].dt.day
total['weekday'] = total['연월일'].dt.weekday

In [ ]:
total

,연월일,시간,구분,공급량,year,month,day,weekday
0,2013-01-01,1,0,2497.129,2013,1,1,1
1,2013-01-01,2,0,2363.265,2013,1,1,1
2,2013-01-01,3,0,2258.505,2013,1,1,1
3,2013-01-01,4,0,2243.969,2013,1,1,1
4,2013-01-01,5,0,2344.105,2013,1,1,1
...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0
368084,2018-12-31,21,6,669.961,2018,12,31,0
368085,2018-12-31,22,6,657.941,2018,12,31,0
368086,2018-12-31,23,6,610.953,2018,12,31,0


## Feature Engineering

In [ ]:
# # 전년도 가스 공급량 feature 추가
# def last_year(x) :
#     alp = x['구분']
#     day = x['연월일']
#     return day-datetime.timedelta(days=1)

# total['last_y'] = total.apply(last_year, axis=1)
# total

,연월일,시간,구분,공급량,year,month,day,weekday,last_y
0,2013-01-01,1,0,2497.129,2013,1,1,1,2012-12-31
1,2013-01-01,2,0,2363.265,2013,1,1,1,2012-12-31
2,2013-01-01,3,0,2258.505,2013,1,1,1,2012-12-31
3,2013-01-01,4,0,2243.969,2013,1,1,1,2012-12-31
4,2013-01-01,5,0,2344.105,2013,1,1,1,2012-12-31
...,...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0,2018-12-30
368084,2018-12-31,21,6,669.961,2018,12,31,0,2018-12-30
368085,2018-12-31,22,6,657.941,2018,12,31,0,2018-12-30
368086,2018-12-31,23,6,610.953,2018,12,31,0,2018-12-30


In [ ]:
# def last_gas(x):
#     if str(x['last_y'])[:4] == '2012':
#         return -1
#     day = x['last_y']
    
#     gas = x[x['연월일'] == day]['공급량']
#     return gas

# total['last_gas'] =  total.apply(lambda, axis=1)
# total

In [ ]:
# tt = total.copy()

In [ ]:
# tt['last_gas'] = tt.groupby('구분')['공급량'].shift(8760)
# tt

In [ ]:
# tt[tt['구분'] == 1]

In [ ]:
# tt[tt['구분'] == 1][:1]

In [ ]:
# tt[tt['구분'] == 1][8760:8761]
# tt[tt['구분'] == 1][17520:17521]
# tt[tt['구분'] == 1][26280:26281]
# tt[tt['구분'] == 1][35040:35041]
# tt[tt['구분'] == 1][43801:43801]

In [ ]:
# total = total.drop('last_y',axis=1)
# total

In [ ]:
# total['last_gas'] = total.groupby('구분')['공급량'].shift(8760)
# total

In [12]:
# # total[total['last_gas']==0]   # 0인 경우 없음
# total['last_gas'] = total['last_gas'].fillna(0)

In [13]:
last_gas = total['last_gas'].values
gas = total['공급량'].values
cate = total['구분'].values
last_gas, gas, cate

(array([  0.   ,   0.   ,   0.   , ..., 506.287, 470.638, 444.618]),
 array([2497.129, 2363.265, 2258.505, ...,  657.941,  610.953,  560.896]),
 array([0, 0, 0, ..., 6, 6, 6]))

In [28]:
len(last_gas)


368088

In [21]:
last_gas[6], last_gas[61320]

(0.0, 2497.129)

In [25]:
gas_cumsum = []
gas_cumcnt = []
for i in range(len(last_gas)):  #
    if last_gas[i] == 0.0 :
        gas_cumsum.append(gas[i])
        gas_cumcnt.append(1)
    elif last_gas[i] != 0.0 and cate[i]==cate[i-61320]:
        sum = gas_cumsum[i-61320] + gas[i]
        cnt = gas_cumsum[i-61320] + 1
        gas_cumsum.append(sum)
        gas_cumcnt.append(cnt)
gas[0],gas[61320], gas[61320*2]

(2497.129, 1677.257, 2228.705)

In [27]:
gas_cumsum[0],gas_cumsum[61320], gas_cumsum[61320*2]

(2497.129, 4174.386, 6403.091)

In [31]:
len(gas_cumsum)

366912

In [30]:
gas_cumsum[368086]

IndexError: ignored

## FE : cum mean

In [9]:
total_origin = total.copy()

In [10]:
total['last_gas'] = total.groupby(['구분','month','day','시간'])['공급량'].transform(lambda x: x.shift(1))

In [11]:
total['gas_cumsum'] = total.groupby(['구분','month','day','시간'])['공급량'].transform(lambda x: x.cumsum().shift(1))

In [12]:
total['gas_cumcnt'] = total.groupby(['구분','month','day','시간'])['공급량'].cumcount()

In [71]:
total.head()

,연월일,시간,구분,공급량,year,month,day,weekday,last_gas,gas_cumsum,gas_cumcnt,gas_cummean
0,2013-01-01,1,0,2497.129,2013,1,1,1,NaN,2497.129,1,2497.129
1,2013-01-01,2,0,2363.265,2013,1,1,1,NaN,2363.265,1,2363.265
2,2013-01-01,3,0,2258.505,2013,1,1,1,NaN,2258.505,1,2258.505
3,2013-01-01,4,0,2243.969,2013,1,1,1,NaN,2243.969,1,2243.969
4,2013-01-01,5,0,2344.105,2013,1,1,1,NaN,2344.105,1,2344.105


In [14]:
def cum_mean(x) :
    if x['gas_cumcnt'] == 0 :
        return x['gas_cumsum']
    else :
        mean = x['gas_cumsum']/x['gas_cumcnt']
        return round(mean,3)

total['gas_cummean'] = total.apply(cum_mean, axis=1)

In [15]:
# pd.set_option('display.max_rows', None)
total[(total['시간']==2) & (total['구분']== 0) & (total['month']==2) & (total['day']==1)]

,연월일,시간,구분,공급량,year,month,day,weekday,last_gas,gas_cumsum,gas_cumcnt,gas_cummean
745,2013-02-01,2,0,1377.905,2013,2,1,4,NaN,NaN,0,NaN
62065,2014-02-01,2,0,1315.257,2014,2,1,5,1377.905,1377.905,1,1377.905
123385,2015-02-01,2,0,1729.313,2015,2,1,6,1315.257,2693.162,2,1346.581
184705,2016-02-01,2,0,1719.409,2016,2,1,0,1729.313,4422.475,3,1474.158
246193,2017-02-01,2,0,1936.561,2017,2,1,2,1719.409,6141.884,4,1535.471
307513,2018-02-01,2,0,2119.548,2018,2,1,3,1936.561,8078.445,5,1615.689


In [16]:
train_years = [2014,2015,2016,2017]  #2013은 2012년도가 없어서 예외
val_years = [2018]

In [17]:
train = total[total['year'].isin(train_years)]
val = total[total['year'].isin(val_years)]

In [18]:
features = ['구분', 'month', 'day', 'weekday', '시간','last_gas', 'gas_cummean']
train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']

In [19]:
train_x

,구분,month,day,weekday,시간,last_gas,gas_cummean
61320,0,1,1,2,1,2497.129,2497.129
61321,0,1,1,2,2,2363.265,2363.265
61322,0,1,1,2,3,2258.505,2258.505
61323,0,1,1,2,4,2243.969,2243.969
61324,0,1,1,2,5,2344.105,2344.105
...,...,...,...,...,...,...,...
306763,6,12,31,6,20,531.796,564.180
306764,6,12,31,6,21,504.268,550.588
306765,6,12,31,6,22,482.476,532.706
306766,6,12,31,6,23,450.284,499.088


# 학습

In [52]:
# d_train = lgb.Dataset(train_x, train_y)
# d_val = lgb.Dataset(val_x, val_y)

# params = {
#     'objective': 'regression',
#     'metric':'mae',
#     'seed':42
# }

# model = lgb.train(params, d_train, 500, d_val, verbose_eval=20, early_stopping_rounds=10)

Training until validation scores don't improve for 10 rounds.
[20]	valid_0's l1: 199.309
[40]	valid_0's l1: 163.32
[60]	valid_0's l1: 160.627
Early stopping, best iteration is:
[51]	valid_0's l1: 160.279


In [51]:
import xgboost as xgb
from xgboost import XGBRegressor

dtrain = xgb.DMatrix(data=train_x, label = train_y)
dval = xgb.DMatrix(data=val_x, label = val_y)
wlist = [(dtrain, 'train'), (dval,'eval')]

params = {
    'learning_rate': 0.05,
    'objective': 'reg:squarederror',
    'metric':'mae', 
    'seed':42
}
 

model = xgb.train( params, dtrain, 1000, evals=wlist, verbose_eval=100, early_stopping_rounds=200)

[0]	train-rmse:1220.4	eval-rmse:1369.56
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 200 rounds.
[100]	train-rmse:197.539	eval-rmse:285.215
[200]	train-rmse:174.816	eval-rmse:280.214
[300]	train-rmse:162.664	eval-rmse:281.175
Stopping. Best iteration:
[186]	train-rmse:176.86	eval-rmse:279.734



# 추론 및 결과 제출

In [23]:
test = pd.read_csv(path+'test.csv')
submission = pd.read_csv(path+'sample_submission.csv')

In [ ]:
test.head()

,일자|시간|구분
0,2019-01-01 01 A
1,2019-01-01 02 A
2,2019-01-01 03 A
3,2019-01-01 04 A
4,2019-01-01 05 A


In [ ]:
submission.head()

,일자|시간|구분,공급량
0,2019-01-01 01 A,0
1,2019-01-01 02 A,0
2,2019-01-01 03 A,0
3,2019-01-01 04 A,0
4,2019-01-01 05 A,0


In [24]:
test['일자'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]

In [25]:
test['일자'] = pd.to_datetime(test['일자'])
test['year'] = test['일자'].dt.year
test['month'] = test['일자'].dt.month
test['day'] = test['일자'].dt.day
test['weekday'] = test['일자'].dt.weekday

In [26]:
test['구분'] = test['구분'].map(d_map)

In [ ]:
total.head()

,연월일,시간,구분,공급량,year,month,day,weekday,last_y,last_gas
0,2013-01-01,1,0,2497.129,2013,1,1,1,2012-12-31,NaN
1,2013-01-01,2,0,2363.265,2013,1,1,1,2012-12-31,NaN
2,2013-01-01,3,0,2258.505,2013,1,1,1,2012-12-31,NaN
3,2013-01-01,4,0,2243.969,2013,1,1,1,2012-12-31,NaN
4,2013-01-01,5,0,2344.105,2013,1,1,1,2012-12-31,NaN


In [91]:
total.head()  # 학습 후 다시 생성한 total

,연월일,시간,구분,공급량,year,month,day,weekday
0,2013-01-01,1,0,2497.129,2013,1,1,1
1,2013-01-01,2,0,2363.265,2013,1,1,1
2,2013-01-01,3,0,2258.505,2013,1,1,1
3,2013-01-01,4,0,2243.969,2013,1,1,1
4,2013-01-01,5,0,2344.105,2013,1,1,1


In [ ]:
# total_drop = total.drop('last_gas',axis=1)
# total_drop

,연월일,시간,구분,공급량,year,month,day,weekday
0,2013-01-01,1,0,2497.129,2013,1,1,1
1,2013-01-01,2,0,2363.265,2013,1,1,1
2,2013-01-01,3,0,2258.505,2013,1,1,1
3,2013-01-01,4,0,2243.969,2013,1,1,1
4,2013-01-01,5,0,2344.105,2013,1,1,1
...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0
368084,2018-12-31,21,6,669.961,2018,12,31,0
368085,2018-12-31,22,6,657.941,2018,12,31,0
368086,2018-12-31,23,6,610.953,2018,12,31,0


In [27]:
total['공급량_x'] = total['공급량']
total = total.drop('공급량',axis=1)
total.rename(columns = {'공급량_x' : '공급량'}, inplace = True)

In [95]:
total.head()

,연월일,시간,구분,year,month,day,weekday,공급량
0,2013-01-01,1,0,2013,1,1,1,2497.129
1,2013-01-01,2,0,2013,1,1,1,2363.265
2,2013-01-01,3,0,2013,1,1,1,2258.505
3,2013-01-01,4,0,2013,1,1,1,2243.969
4,2013-01-01,5,0,2013,1,1,1,2344.105


In [97]:
# pd.set_option('display.max_rows', 10)

In [28]:
test_1 = test.drop('일자|시간|구분', axis=1)
test_1.rename(columns = {'일자' : '연월일'}, inplace = True)
test_1['공급량'] = 0

test_1

,연월일,시간,구분,year,month,day,weekday,공급량
0,2019-01-01,1,0,2019,1,1,1,0
1,2019-01-01,2,0,2019,1,1,1,0
2,2019-01-01,3,0,2019,1,1,1,0
3,2019-01-01,4,0,2019,1,1,1,0
4,2019-01-01,5,0,2019,1,1,1,0
...,...,...,...,...,...,...,...,...
15115,2019-03-31,20,6,2019,3,31,6,0
15116,2019-03-31,21,6,2019,3,31,6,0
15117,2019-03-31,22,6,2019,3,31,6,0
15118,2019-03-31,23,6,2019,3,31,6,0


In [31]:
test_2 = pd.concat([total_origin, test_1], axis=0)

In [32]:
test_2[(test_2['시간']==2) & (test_2['구분']== 0) & (test_2['month']==2) & (test_2['day']==1)]

,연월일,시간,구분,공급량,year,month,day,weekday
745,2013-02-01,2,0,1377.905,2013,2,1,4
62065,2014-02-01,2,0,1315.257,2014,2,1,5
123385,2015-02-01,2,0,1729.313,2015,2,1,6
184705,2016-02-01,2,0,1719.409,2016,2,1,0
246193,2017-02-01,2,0,1936.561,2017,2,1,2
307513,2018-02-01,2,0,2119.548,2018,2,1,3
745,2019-02-01,2,0,0.000,2019,2,1,4


In [33]:
test_2['last_gas'] = test_2.groupby(['구분','month','day','시간'])['공급량'].transform(lambda x: x.shift(1))
test_2['gas_cumsum'] = test_2.groupby(['구분','month','day','시간'])['공급량'].transform(lambda x: x.cumsum().shift(1))
test_2['gas_cumcnt'] = test_2.groupby(['구분','month','day','시간'])['공급량'].cumcount()

def cum_mean(x) :
    if x['gas_cumcnt'] == 0 :
        return x['gas_cumsum']
    else :
        mean = x['gas_cumsum']/x['gas_cumcnt']
        return round(mean,3)
test_2['gas_cummean'] = test_2.apply(cum_mean, axis=1)

In [34]:
test_3 = test_2[len(total_origin):]

In [35]:
test_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15120 entries, 0 to 15119
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   연월일          15120 non-null  datetime64[ns]
 1   시간           15120 non-null  int64         
 2   구분           15120 non-null  int64         
 3   공급량          15120 non-null  float64       
 4   year         15120 non-null  int64         
 5   month        15120 non-null  int64         
 6   day          15120 non-null  int64         
 7   weekday      15120 non-null  int64         
 8   last_gas     15120 non-null  float64       
 9   gas_cumsum   15120 non-null  float64       
 10  gas_cumcnt   15120 non-null  int64         
 11  gas_cummean  15120 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(7)
memory usage: 1.5 MB


In [ ]:
# test_3['last_gas'] = test_3['last_gas'].astype(float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
test_x = test_3[features]

In [37]:
test_x

,구분,month,day,weekday,시간,last_gas,gas_cummean
0,0,1,1,1,1,1765.008,1913.798
1,0,1,1,1,2,1679.186,1801.092
2,0,1,1,1,3,1610.885,1696.859
3,0,1,1,1,4,1604.123,1689.183
4,0,1,1,1,5,1711.506,1796.721
...,...,...,...,...,...,...,...
15115,6,3,31,6,20,244.162,316.634
15116,6,3,31,6,21,248.059,327.583
15117,6,3,31,6,22,231.181,311.250
15118,6,3,31,6,23,199.022,277.962


In [38]:
#xgboost 실행을 위한 코드
test_x = xgb.DMatrix(test_x)

In [39]:
preds = model.predict(test_x)

In [40]:
submission['공급량'] = preds

In [41]:
submission

,일자|시간|구분,공급량
0,2019-01-01 01 A,1721.539429
1,2019-01-01 02 A,1523.760132
2,2019-01-01 03 A,1422.074951
3,2019-01-01 04 A,1414.206421
4,2019-01-01 05 A,1527.060913
...,...,...
15115,2019-03-31 20 H,286.525177
15116,2019-03-31 21 H,301.571869
15117,2019-03-31 22 H,279.252869
15118,2019-03-31 23 H,244.142807


In [42]:
submission.to_csv('submission6.csv', index=False)